## Amazon Sagemaker Clarify 01
### Fairness and Explainability
- AWS Docs Link: https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html
- ReadTheDocs Link: https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-clarify/fairness_and_explainability/fairness_and_explainability.html

Amazon SageMaker Clarify helps improve your machine learning models by detecting potential bias and helping explain how these models make predictions. The fairness and explainability functionality provided by SageMaker Clarify takes a step towards enabling AWS customers to build trustworthy and understandable machine learning models. The product comes with the tools to help you with the following tasks.

Measure biases that can occur during each stage of the ML lifecycle (data collection, model training and tuning, and monitoring of ML models deployed for inference).

Generate model governance reports targeting risk and compliance teams and external regulators.

Provide explanations of the data, models, and monitoring used to assess predictions.

This sample notebook walks you through:
1. Key terms and concepts needed to understand SageMaker Clarify
2. Measuring the pre-training bias of a dataset and post-training bias of a model
3. Explaining the importance of the various input features on the model’s decision
4. Accessing the reports through SageMaker Studio if you have an instance set up.

   
In doing so, the notebook first trains a SageMaker XGBoost model using training dataset, then use Amazon SageMaker Python SDK to launch SageMaker Clarify jobs to analyze an example dataset in CSV format. In more details:

- Create an Instance of Sagemaker Clarify, specify the EC2 instance type, count, execution role, and sagemaker session.
- Now define some configurations on
  - DataConfig: how data is defined
  - ModelConfig: how model shall take input-output at what instance the model should be hosted to make inferences,
  - ModelPredictedLabelConfig: what should be the tresholds for probability to assign labels.
  - BiasConfig: For what factor you think bias needs to be analyzed.
- An output report will be generated.


#### Important Notes
1. The datatype of the target label in train/test data should be same, otherwise when you run the clarify process it will raise an error. The clarify will get the predictions and assigne the integer labels, so the label in the data should also be in integer.

In [1]:
import pandas as pd    
import numpy as np    
import boto3   
import sagemaker  

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
ROLE = sagemaker.get_execution_role()
print(ROLE)

boto_session = boto3.session.Session()
REGION = boto_session.region_name
print(REGION)

sagemaker_session = sagemaker.session.Session(boto_session=boto_session)
BUCKET = sagemaker_session.default_bucket()
print(BUCKET)

s3_boto_client = boto_session.client("s3")
sagemaker_boto_client = boto_session.client("sagemaker")

PREFIX_KEY = "DEMO-sagemaker-clarify"


arn:aws:iam::205930620783:role/service-role/AmazonSageMaker-ExecutionRole-20250401T145997
us-east-1
sagemaker-us-east-1-205930620783


In [3]:

train_uri = f"s3://sagemaker-us-east-1-205930620783/DEMO-sagemaker-clarify/data/train_preocessed.csv"

train_data = pd.read_csv(train_uri)
FEATURES = train_data.columns.tolist()
TARGET = FEATURES[0]
#FEATURES = FEATURES[1:]
print(TARGET, " : ", FEATURES)
print(train_data[TARGET].value_counts())
print(train_data[TARGET].isna().sum())
#print(train_data.dtypes)

print(train_data[TARGET].dtype)
if train_data[TARGET].dtype != 'int':
    print("Convert to int")
    train_data[TARGET] = train_data[TARGET].astype('int')
    train_data.to_csv("data/train_preocessed.csv", index=False)
    from sagemaker.s3 import S3Uploader
    train_uri = S3Uploader.upload("data/train_preocessed.csv", "s3://{}/{}/data".format(BUCKET, PREFIX_KEY))
    #test_uri = S3Uploader.upload("test_features.csv", "s3://{}/{}".format(bucket, prefix))
    print(train_uri)

Target_>50K  :  ['Target_>50K', 'Age', 'fnlwgt', 'Education-Num', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Workclass_Federal-gov', 'Workclass_Local-gov', 'Workclass_Never-worked', 'Workclass_Private', 'Workclass_Self-emp-inc', 'Workclass_Self-emp-not-inc', 'Workclass_State-gov', 'Workclass_Without-pay', 'Education_11th', 'Education_12th', 'Education_1st-4th', 'Education_5th-6th', 'Education_7th-8th', 'Education_9th', 'Education_Assoc-acdm', 'Education_Assoc-voc', 'Education_Bachelors', 'Education_Doctorate', 'Education_HS-grad', 'Education_Masters', 'Education_Preschool', 'Education_Prof-school', 'Education_Some-college', 'Marital Status_Married-AF-spouse', 'Marital Status_Married-civ-spouse', 'Marital Status_Married-spouse-absent', 'Marital Status_Never-married', 'Marital Status_Separated', 'Marital Status_Widowed', 'Occupation_Adm-clerical', 'Occupation_Armed-Forces', 'Occupation_Craft-repair', 'Occupation_Exec-managerial', 'Occupation_Farming-fishing', 'Occupation_Handlers

### Create a sagemaker clarify processor

In [4]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(role=ROLE, instance_count=1, instance_type="ml.m5.xlarge", sagemaker_session=sagemaker_session)

[06/01/25 11:07:49] INFO     Ignoring unnecessary instance type: None.                            ]8;id=407112;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=585887;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#530\530]8;;\

## Detecting Bias
SageMaker Clarify helps you detect possible pre-training and post-training biases using a variety of metrics.

### Writing DataConfig
A DataConfig object communicates some basic information about data I/O to SageMaker Clarify. For our example here we provide the below information:


In [5]:
bias_report_output_path = "s3://{}/{}/clarify-bias".format(BUCKET, PREFIX_KEY)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_uri,
    s3_output_path=bias_report_output_path,
    label=TARGET,
    headers=FEATURES,
    dataset_type="text/csv",
)

### Writing ModelConfig
A ModelConfig object communicates information about your trained model. To avoid additional traffic to the production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. For our example here we provide the below information:
Sagemaker clarify will use the input features to predict the outcomes and will see if model predictios are bias towards any feature like gender, race.

-  model_name: name of the concerned model, using name of the xgboost model trained earlier
- instance_type and initial_instance_count specify your preferred instance type and instance count used to run your model on during SageMaker Clarify’s processing. The example dataset is small, so a single standard instance is good enough to run this example.
- accept_type denotes the endpoint response payload format, and content_type denotes the payload format of request to the endpoint. As per the example model we created above both of these will be text/csv.


In [7]:

model_config = clarify.ModelConfig(
    model_name="Model-Adult-Dataset-31-05-2025-18-20-58",
    instance_type="ml.c5.xlarge",
    instance_count=1,
    accept_type="text/csv", # Endpoint Reponse Type
    content_type="text/csv", # Endpoint Input Type
)

### Writing ModelPredictedLabelConfig
A ModelPredictedLabelConfig provides information on the format of your predictions. XGBoost model outputs probabilities of samples, so SageMaker Clarify invokes the endpoint then uses probability_threshold to convert the probability to binary labels for bias analysis. Prediction above the threshold is interpreted as label value 1 and below or equal as label value 0.


In [8]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

### Writing BiasConfig
BiasConfig contains configuration values for detecting bias using a Clarify container.

For our demo we provide the following information in BiasConfig API:
- label_values_or_threshold: List of label value(s) or threshold to indicate positive outcome used for bias metrics. Here positive outcome is earning >$50,000.
- facet_name: Sensitive columns of the dataset, “Sex” is the category
- facet_values_or_threshold: values of the sensitive group, “Female” respondents are the sensitive group.
- group_name: This example has selected the “Age” column which is used to form subgroups for the measurement of bias metric Conditional Demographic Disparity (CDD) or Conditional Demographic Disparity in Predicted Labels (CDDPL).

SageMaker Clarify can handle both categorical and continuous data for facet: values_or_threshold and for label_values_or_threshold. In this case we are using categorical data. 

**The results will show if the model has a preference for records of one sex over the other.**

In [9]:

bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="Sex_Male", facet_values_or_threshold=[0], group_name="Age"
)

#### Pre-training Bias
Bias can be present in your data before any model training occurs. Inspecting your data for bias before training begins can help detect any data collection gaps, inform your feature engineering, and help you understand what societal biases the data may reflect.

Computing pre-training bias metrics does not require a trained model.

#### Post-training Bias
Computing post-training bias metrics does require a trained model.

Unbiased training data (as determined by concepts of fairness measured by bias metric) may still result in biased model predictions after training. Whether this occurs depends on several factors including hyperparameter choices.

You can run these options separately with run_pre_training_bias() and run_post_training_bias() or at the same time with run_bias() as shown below. We use following additional parameters for the api call:

- pre_training_methods: Pre-training bias metrics to be computed. The detailed description of the metrics can be found on Measure Pre-training Bias. This example sets methods to “all” to compute all the pre-training bias metrics.
- post_training_methods: Post-training bias metrics to be computed. The detailed description of the metrics can be found on Measure Post-training Bias. This example sets methods to “all” to compute all the post-training bias metrics.

In [ ]:
# The job takes about 10 minutes to run
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

[06/01/25 07:10:04] INFO     Analysis Config: {'dataset_type': 'text/csv', 'headers':               ]8;id=379558;file:///opt/conda/lib/python3.12/site-packages/sagemaker/clarify.py\clarify.py]8;;\:]8;id=34359;file:///opt/conda/lib/python3.12/site-packages/sagemaker/clarify.py#1987\1987]8;;\
                             ['Target_>50K', 'Age', 'fnlwgt', 'Education-Num', 'Capital Gain',                     
                             'Capital Loss', 'Hours per week', 'Workclass_Federal-gov',                            
                             'Workclass_Local-gov', 'Workclass_Never-worked', 'Workclass_Private',                 
                             'Workclass_Self-emp-inc', 'Workclass_Self-emp-not-inc',                               
                             'Workclass_State-gov', 'Workclass_Without-pay', 'Education_11th',                     
                             'Education_12th', 'Education_1st-4th', 'Education_5th-6th',                           
                             'Education_7th-8th', 'Education_9th', 'Education_Assoc-acdm',                         
                             'Education_Assoc-voc', 'Education_Bachelors', 'Education_Doctorate',                  
                             'Education_HS-grad', 'Education_Masters', 'Education_Preschool',                      
                             'Education_Prof-school', 'Education_Some-college', 'Marital                           
                             Status_Married-AF-spouse', 'Marital Status_Married-civ-spouse',                       
                             'Marital Status_Married-spouse-absent', 'Marital                                      
                             Status_Never-married', 'Marital Status_Separated', 'Marital                           
                             Status_Widowed', 'Occupation_Adm-clerical', 'Occupation_Armed-Forces',                
                             'Occupation_Craft-repair', 'Occupation_Exec-managerial',                              
                             'Occupation_Farming-fishing', 'Occupation_Handlers-cleaners',                         
                             'Occupation_Machine-op-inspct', 'Occupation_Other-service',                           
                             'Occupation_Priv-house-serv', 'Occupation_Prof-specialty',                            
                             'Occupation_Protective-serv', 'Occupation_Sales',                                     
                             'Occupation_Tech-support', 'Occupation_Transport-moving',                             
                             'Relationship_Not-in-family', 'Relationship_Other-relative',                          
                             'Relationship_Own-child', 'Relationship_Unmarried',                                   
                             'Relationship_Wife', 'Ethnic group_Asian-Pac-Islander', 'Ethnic                       
                             group_Black', 'Ethnic group_Other', 'Ethnic group_White', 'Sex_Male',                 
                             'Country_Cambodia', 'Country_Canada', 'Country_China',                                
                             'Country_Columbia', 'Country_Cuba', 'Country_Dominican-Republic',                     
                             'Country_Ecuador', 'Country_El-Salvador', 'Country_England',                          
                             'Country_France', 'Country_Germany', 'Country_Greece',                                
                             'Country_Guatemala', 'Country_Haiti', 'Country_Holand-Netherlands',                   
                             'Country_Honduras', 'Country_Hong', 'Country_Hungary',                                
                             'Country_India', 'Country_Iran', 'Country_Ireland', 'Country_Italy',                  
                             'Country_Jamaica', 'Country_Japan', 'Country_Laos', 'Country_Mexico',      

[06/01/25 07:10:05] INFO     Creating processing-job with name Clarify-Bias-2025-06-01-07-10-04-980 ]8;id=73196;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=383402;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#1575\1575]8;;\

........

## Explaining Predictions
There are expanding business needs and legislative regulations that require explanations of why a model made the decision it did. SageMaker Clarify uses Kernel SHAP to explain the contribution that each input feature makes to the final decision.

For run_explainability API call we need similar DataConfig and ModelConfig objects we defined above. SHAPConfig here is the config class for Kernel SHAP algorithm.

For our demo we pass the following information in SHAPConfig:
- baseline: Kernel SHAP algorithm requires a baseline (also known as background dataset). If not provided, a baseline is calculated automatically by SageMaker Clarify using K-means or K-prototypes in the input dataset. Baseline dataset type shall be the same as dataset_type, and baseline samples shall only include features. By definition, baseline should either be a S3 URI to the baseline dataset file, or an in-place list of samples. In this case we chose the latter, and put the mean of the train dataset to the list. For more details on baseline selection please refer this documentation.
- num_samples: Number of samples to be used in the Kernel SHAP algorithm. This number determines the size of the generated synthetic dataset to compute the SHAP values.
- agg_method: Aggregation method for global SHAP values. For our example here we are using mean_abs i.e. mean of absolute SHAP values for all instances
- save_local_shap_values: Indicates whether to save the local SHAP values in the output location. Default is True.


In [11]:

explainability_output_path = "s3://{}/{}/clarify-explainability".format(BUCKET, PREFIX_KEY)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=train_uri,
    s3_output_path=explainability_output_path,
    label=TARGET,
    headers=FEATURES,
    dataset_type="text/csv",
)

baseline = [train_data.mean().iloc[1:].values.tolist()]
shap_config = clarify.SHAPConfig(
    baseline=baseline,
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)
# The job takes about 10 minutes to run
clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

[06/01/25 11:08:32] INFO     Analysis Config: {'dataset_type': 'text/csv', 'headers':               ]8;id=232133;file:///opt/conda/lib/python3.12/site-packages/sagemaker/clarify.py\clarify.py]8;;\:]8;id=573802;file:///opt/conda/lib/python3.12/site-packages/sagemaker/clarify.py#1987\1987]8;;\
                             ['Target_>50K', 'Age', 'fnlwgt', 'Education-Num', 'Capital Gain',                     
                             'Capital Loss', 'Hours per week', 'Workclass_Federal-gov',                            
                             'Workclass_Local-gov', 'Workclass_Never-worked', 'Workclass_Private',                 
                             'Workclass_Self-emp-inc', 'Workclass_Self-emp-not-inc',                               
                             'Workclass_State-gov', 'Workclass_Without-pay', 'Education_11th',                     
                             'Education_12th', 'Education_1st-4th', 'Education_5th-6th',                           
                             'Education_7th-8th', 'Education_9th', 'Education_Assoc-acdm',                         
                             'Education_Assoc-voc', 'Education_Bachelors', 'Education_Doctorate',                  
                             'Education_HS-grad', 'Education_Masters', 'Education_Preschool',                      
                             'Education_Prof-school', 'Education_Some-college', 'Marital                           
                             Status_Married-AF-spouse', 'Marital Status_Married-civ-spouse',                       
                             'Marital Status_Married-spouse-absent', 'Marital                                      
                             Status_Never-married', 'Marital Status_Separated', 'Marital                           
                             Status_Widowed', 'Occupation_Adm-clerical', 'Occupation_Armed-Forces',                
                             'Occupation_Craft-repair', 'Occupation_Exec-managerial',                              
                             'Occupation_Farming-fishing', 'Occupation_Handlers-cleaners',                         
                             'Occupation_Machine-op-inspct', 'Occupation_Other-service',                           
                             'Occupation_Priv-house-serv', 'Occupation_Prof-specialty',                            
                             'Occupation_Protective-serv', 'Occupation_Sales',                                     
                             'Occupation_Tech-support', 'Occupation_Transport-moving',                             
                             'Relationship_Not-in-family', 'Relationship_Other-relative',                          
                             'Relationship_Own-child', 'Relationship_Unmarried',                                   
                             'Relationship_Wife', 'Ethnic group_Asian-Pac-Islander', 'Ethnic                       
                             group_Black', 'Ethnic group_Other', 'Ethnic group_White', 'Sex_Male',                 
                             'Country_Cambodia', 'Country_Canada', 'Country_China',                                
                             'Country_Columbia', 'Country_Cuba', 'Country_Dominican-Republic',                     
                             'Country_Ecuador', 'Country_El-Salvador', 'Country_England',                          
                             'Country_France', 'Country_Germany', 'Country_Greece',                                
                             'Country_Guatemala', 'Country_Haiti', 'Country_Holand-Netherlands',                   
                             'Country_Honduras', 'Country_Hong', 'Country_Hungary',                                
                             'Country_India', 'Country_Iran', 'Country_Ireland', 'Country_Italy',                  
                             'Country_Jamaica', 'Country_Japan', 'Country_Laos', 'Country_Mexico',     

[06/01/25 11:08:33] INFO     Creating processing-job with name                                      ]8;id=423502;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=507916;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#1575\1575]8;;\
                             Clarify-Explainability-2025-06-01-11-08-32-974                                        

..................sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
We are not in a supported iso region, /bin/sh exiting gracefully with no changes.
INFO:sagemaker-clarify-processing:Starting SageMaker Clarify Processing job
INFO:analyzer.data_loading.data_loader_util:Analysis config path: /opt/ml/processing/input/config/analysis_config.json
INFO:analyzer.data_loading.data_loader_util:Analysis result path: /opt/ml/processing/output
INFO:analyzer.data_loading.data_loader_util:This host is algo-1.
INFO:analyzer.data_loading.data_loader_util:This host is the leader.
INFO:analyzer.data_loading.data_loader_util:Number of hosts in the cluster is 1.
INFO:sagemaker-clarify-processing:Running Python / Pandas based analyzer.
INFO:analyzer.data_loading.data_loader_factory:Dataset type: text/csv uri: /opt/ml/processing/input/data
INFO:sagemaker-clarify

## Analysis of local explanations
It is possible to visualize the local explanations for single examples in your dataset. You can use the obtained results from running Kernel SHAP algorithm for global explanations.

You can simply load the local explanations stored in your output path, and visualize the explanation (i.e., the impact that the single features have on the prediction of your model) for any single example.


In [22]:
local_explanations_out = pd.read_csv(explainability_output_path + "/explanations_shap/out.csv")
feature_names = [str.replace(c, "_label0", "") for c in local_explanations_out.columns.to_series()]
local_explanations_out.columns = feature_names

In [23]:
print(local_explanations_out.shape)

(32561, 100)


In [31]:
expl_values = local_explanations_out.iloc[selected_example]
expl_values = pd.concat([expl_values, np.abs(expl_values)], axis=1)
expl_values.columns=['value','abs_value']
expl_values.sort_values('abs_value', ascending=False).head(10)

,value,abs_value
Country_Vietnam,0.017842,0.017842
Country_Hong,0.017842,0.017842
Marital Status_Widowed,0.017842,0.017842
Education_1st-4th,0.017842,0.017842
Education_Assoc-acdm,0.017842,0.017842
Country_Iran,0.017842,0.017842
Relationship_Unmarried,0.017842,0.017842
fnlwgt,0.017842,0.017842
Country_Thailand,0.017842,0.017842
Country_Puerto-Rico,0.017842,0.017842


In [32]:
selected_example = 111
print(
    "Example number:",
    selected_example,
    "\nwith model prediction:",
    sum(local_explanations_out.iloc[selected_example]),
)
print("\nFeature values -- Label", train_data.iloc[selected_example])
#local_explanations_out.iloc[selected_example].plot(
#    kind="bar", title="Local explanation for the example number " + str(selected_example), rot=90
#)

Example number: 111 
with model prediction: 0.8422952950932054

Feature values -- Label Target_>50K                1.000000
Age                       -0.042642
fnlwgt                    -1.179122
Education-Num              1.912138
Capital Gain              -0.145920
                             ...   
Country_Thailand           0.000000
Country_Trinadad&Tobago    0.000000
Country_United-States      1.000000
Country_Vietnam            0.000000
Country_Yugoslavia         0.000000
Name: 111, Length: 101, dtype: float64


## Delete the Model and clean the resources

In [19]:
model_name = "sagemaker-xgboost-2025-06-01-05-35-35-872" 
sagemaker_session.delete_model(model_name)

In [20]:
!aws s3 cp s3://sagemaker-us-east-1-205930620783/DEMO-sagemaker-clarify/ ./s3_data/ --recursive 


download: s3://sagemaker-us-east-1-205930620783/DEMO-sagemaker-clarify/clarify-bias/analysis_config.json to s3_data/clarify-bias/analysis_config.json
download: s3://sagemaker-us-east-1-205930620783/DEMO-sagemaker-clarify/clarify-bias/analysis.json to s3_data/clarify-bias/analysis.json
download: s3://sagemaker-us-east-1-205930620783/DEMO-sagemaker-clarify/clarify-explainability/analysis_config.json to s3_data/clarify-explainability/analysis_config.json
download: s3://sagemaker-us-east-1-205930620783/DEMO-sagemaker-clarify/clarify-explainability/explanations_shap/baseline.csv to s3_data/clarify-explainability/explanations_shap/baseline.csv
download: s3://sagemaker-us-east-1-205930620783/DEMO-sagemaker-clarify/clarify-explainability/analysis.json to s3_data/clarify-explainability/analysis.json
download: s3://sagemaker-us-east-1-205930620783/DEMO-sagemaker-clarify/clarify-explainability/report.ipynb to s3_data/clarify-explainability/report.ipynb
download: s3://sagemaker-us-east-1-205930620